# <font color='blue'>Projeto Spark Streaming</font>
# <font color='blue'>Análise de sentimentos em tempo real com Spark em Python 3</font>



****** Em  caso de receber  mensagem de erro "name 'sc' is not defined", reinice o pyspark e apagando o diretório metastore_db no mesmo diretório onde está este Jupyter notebook ******

http://localhost:4040 sempre que quiser acompanhar a execução dos jobs

## Spark Streaming - Twitter

In [ ]:
# Instalando pacotes necessários no python
!pip install requests_oauthlib
!pip install twython
!pip install nltk

In [ ]:
# IMportando Módulos necessários
from pyspark.streaming import StreamingContext
from pyspark import SparkContext
from requests_oauthlib import OAuth1Session
from operator import add
import requests_oauthlib
from time import gmtime, strftime
import requests
import time
import string
import ast
import json

In [ ]:
# Importando o NLTK
import nltk
from nltk.classify import NaiveBayesClassifier
from nltk.sentiment import SentimentAnalyzer
from nltk.corpus import subjectivity
from nltk.corpus import stopwords
from nltk.sentiment.util import *

In [ ]:
# Intervalo de captura dos tweets
INTERVALO_BATCH = 5

In [ ]:
# Criando um contexto no spark - StreamingContext
ssc = StreamingContext(sc, INTERVALO_BATCH)

## Treinando o Classificador de Análise de Sentimento

Utlizaremos um dataset de treino fornecido pela Universidade de Michigan, para competições do Kaggle --> https://inclass.kaggle.com/c/si650winter11.

dataset com 1,578,627 tweets classificados como: 

### 1 para o sentimento positivo 
### 0 para o sentimento negativo 

In [ ]:
# criando um RDD em memória com Spark
arquivo = sc.textFile("/Users/davi/Projeto-Python-Spark/dataset_analise_sentimento.csv")

In [ ]:
# Retirando cabeçalho do dataset
header = arquivo.take(1)[0]
dataset = arquivo.filter(lambda line: line != header)

In [ ]:
type(dataset)

pyspark.rdd.PipelinedRDD

In [ ]:
# função de pré-processamento dos dados, separando as colunas em cada linha, criando uma tupla e removendo a pontuação.
def get_row(line):
  row = line.split(',')
  sentimento = row[1]
  tweet = row[3].strip()
  translator = str.maketrans({key: None for key in string.punctuation})
  tweet = tweet.translate(translator)
  tweet = tweet.split(' ')
  tweet_lower = []
  for word in tweet:
    tweet_lower.append(word.lower())
  return (tweet_lower, sentimento)

In [ ]:
# Aplicando a função map a cada linha do dataset, como os RDD ´s são imutáveis necessitamos salvar a operação em outro dataset de treino
dataset_treino = dataset.map(lambda line: get_row(line))

In [ ]:
# Criando um objeto SentimentAnalyzer - analisador de sentimentos 
analisador_de_sentimentos = SentimentAnalyzer()

In [ ]:
# Baixando as stop words em inglês - cerca de 5GB
# https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
# nltk.download()
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to /Users/dmpm/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
# Carregando a lista de stopwords em Inglês 
stopwords_all = []
for word in stopwords.words('english'):
  stopwords_all.append(word)
  stopwords_all.append(word + '_NEG')

In [ ]:
# Carregando 10.000 tweets do dataset de treino e retorna todas as palavras que não são stopwords
dataset_treino_amostra = dataset_treino.take(10000)

In [ ]:
all_words_neg = analisador_de_sentimentos.all_words([mark_negation(doc) for doc in dataset_treino_amostra])
all_words_neg_nostops = [x for x in all_words_neg if x not in stopwords_all]

In [ ]:
# Cria um unigram (n-grama) e extrai as features
unigram_feats = analisador_de_sentimentos.unigram_word_feats(all_words_neg_nostops, top_n = 200)
analisador_de_sentimentos.add_feat_extractor(extract_unigram_feats, unigrams = unigram_feats)
training_set = analisador_de_sentimentos.apply_features(dataset_treino_amostra)

In [ ]:
type(training_set)

nltk.collections.LazyMap

In [ ]:
print(training_set)

[({'contains()': False, 'contains(im)': False, 'contains(_NEG)': False, 'contains(followfriday)': False, 'contains(amp)': False, 'contains(dont)': False, 'contains(day)': False, 'contains(love)': False, 'contains(like)': False, 'contains(cant)': False, 'contains(good)': False, 'contains(get)': False, 'contains(go)': False, 'contains(today)': False, 'contains(got)': False, 'contains(want)': False, 'contains(time)': False, 'contains(going)': False, 'contains(back)': False, 'contains(one)': False, 'contains(sad)': True, 'contains(really)': False, 'contains(miss)': False, 'contains(u)': False, 'contains(work)': False, 'contains(new)': False, 'contains(2)': False, 'contains(last)': False, 'contains(still)': False, 'contains(twitter)': False, 'contains(night)': False, 'contains(great)': False, 'contains(lol)': False, 'contains(follow)': False, 'contains(need)': False, 'contains(see)': False, 'contains(much)': False, 'contains(myweakness)': False, 'contains(get_NEG)': False, 'contains(didnt)'

In [ ]:
# Treinando o modelo no MLib
modelo_treino = NaiveBayesClassifier.train
classificador = analisador_de_sentimentos.train(modelo_treino, training_set)

Training classifier


In [ ]:
# Testando o classificador 
test_sentence1 = [(['this', 'program', 'is', 'bad'], '')]
test_sentence2 = [(['tough', 'day', 'at', 'work', 'today'], '')]
test_sentence3 = [(['good', 'wonderful', 'amazing', 'awesome'], '')]
test_set = analisador_de_sentimentos.apply_features(test_sentence1)
test_set2 = analisador_de_sentimentos.apply_features(test_sentence2)
test_set3 = analisador_de_sentimentos.apply_features(test_sentence3)

In [ ]:
# Autenticando no Twitter, devem ser utlizadas os tokens de acesso de cada desenvolvedor, por isto estão ocultas 
consumer_key = "xxx"
consumer_secret = "xxx"
access_token = "xxx"
access_token_secret = "xxx"

In [ ]:
# Especificando a URLde busca
search_term = 'Trump'
sample_url = 'https://stream.twitter.com/1.1/statuses/sample.json'
filter_url = 'https://stream.twitter.com/1.1/statuses/filter.json?track='+search_term

In [ ]:
# Criando o objeto de atutenticação para o Twitter
auth = requests_oauthlib.OAuth1(consumer_key, consumer_secret, access_token, access_token_secret)

In [ ]:
# Configurando o Stream no spark-streaming
rdd = ssc.sparkContext.parallelize([0])
stream = ssc.queueStream([], default = rdd)

In [ ]:
type(stream)

pyspark.streaming.dstream.DStream

In [ ]:
# tweets por atualização
NUM_TWEETS = 500  

In [ ]:
# conectando ao Twitter e retornando um número específico de Tweets (NUM_TWEETS), carregando-os em um JSON
def tfunc(t, rdd):
  return rdd.flatMap(lambda x: stream_twitter_data())

def stream_twitter_data():
  response = requests.get(filter_url, auth = auth, stream = True)
  print(filter_url, response)
  count = 0
  for line in response.iter_lines():
    try:
      if count > NUM_TWEETS:
        break
      post = json.loads(line.decode('utf-8'))
      contents = [post['text']]
      count += 1
      yield str(contents)
    except:
      result = False

In [ ]:
stream = stream.transform(tfunc)

In [ ]:
coord_stream = stream.map(lambda line: ast.literal_eval(line))

In [ ]:
# Essa função classifica os tweets, aplicando as features do modelo criado anteriormente
def classifica_tweet(tweet):
  sentence = [(tweet, '')]
  test_set = analisador_de_sentimentos.apply_features(sentence)
  print(tweet, classificador.classify(test_set[0][0]))
  return(tweet, classificador.classify(test_set[0][0]))

In [ ]:
# Essa função retorna o texto dos tweets
def get_tweet_text(rdd):
  for line in rdd:
    tweet = line.strip()
    translator = str.maketrans({key: None for key in string.punctuation})
    tweet = tweet.translate(translator)
    tweet = tweet.split(' ')
    tweet_lower = []
    for word in tweet:
      tweet_lower.append(word.lower())
    return(classifica_tweet(tweet_lower))

In [ ]:
# Criando uma lista vazia para os resultados
resultados = []

In [ ]:
# salvando com uma função o resultado dos batches de Tweets junto com o timestamp
def output_rdd(rdd):
  global resultados
  pairs = rdd.map(lambda x: (get_tweet_text(x)[1],1))
  counts = pairs.reduceByKey(add)
  output = []
  for count in counts.collect():
    output.append(count)
  result = [time.strftime("%I:%M:%S"), output]
  resultados.append(result)
  print(result)

In [ ]:
# foreachRDD() aplica uma função a cada RDD do streaming de dados
coord_stream.foreachRDD(lambda t, rdd: output_rdd(rdd))

In [ ]:
# inciando o  streamingno spark-streaming
ssc.start()
# ssc.awaitTermination()

['10:40:22', []]
['10:40:54', [('0', 372), ('1', 129)]]


In [ ]:
cont = True
while cont:
  if len(resultados) > 5:
    cont = False

['10:41:24', [('0', 389), ('1', 112)]]
['10:41:55', [('0', 371), ('1', 130)]]
['10:42:26', [('1', 131), ('0', 370)]]
['10:42:57', [('1', 138), ('0', 363)]]
['10:43:29', [('1', 127), ('0', 374)]]
['10:43:59', [('0', 375), ('1', 126)]]
['10:44:30', [('1', 122), ('0', 379)]]
['10:45:01', [('1', 131), ('0', 370)]]
['10:45:33', [('0', 368), ('1', 133)]]
['10:46:04', [('1', 124), ('0', 377)]]
['10:46:35', [('1', 120), ('0', 381)]]
['10:47:06', [('0', 387), ('1', 114)]]


In [ ]:
# Gravando e salvando os resultados
rdd_save = '/Users/davi/Projeto-Python-Spark/dataset_analise_sentimento.csv/r'+time.strftime("%I%M%S")
resultados_rdd = sc.parallelize(resultados)
resultados_rdd.saveAsTextFile(rdd_save)

['10:47:36', [('0', 386), ('1', 115)]]


In [ ]:
# Visualizando os resultados do RDD
resultados_rdd.collect()

[['10:40:22', []],
 ['10:40:54', [('0', 372), ('1', 129)]],
 ['10:41:24', [('0', 389), ('1', 112)]],
 ['10:41:55', [('0', 371), ('1', 130)]],
 ['10:42:26', [('1', 131), ('0', 370)]],
 ['10:42:57', [('1', 138), ('0', 363)]],
 ['10:43:29', [('1', 127), ('0', 374)]],
 ['10:43:59', [('0', 375), ('1', 126)]],
 ['10:44:30', [('1', 122), ('0', 379)]],
 ['10:45:01', [('1', 131), ('0', 370)]],
 ['10:45:33', [('0', 368), ('1', 133)]],
 ['10:46:04', [('1', 124), ('0', 377)]],
 ['10:46:35', [('1', 120), ('0', 381)]],
 ['10:47:06', [('0', 387), ('1', 114)]]]

['10:48:10', [('0', 375), ('1', 126)]]
['10:48:41', [('0', 378), ('1', 123)]]
['10:49:11', [('0', 391), ('1', 110)]]
['10:49:43', [('0', 385), ('1', 116)]]


In [ ]:
# parando o streaming
ssc.stop()

#Referências:

https://www.datascienceacademy.com.br/path-player?courseid=analise-de-dados-com-python&unit=5d1d100a5e4cde60708b457dUnit

Hadoop: The Definitive Guide,
White Tom

Natural Language Processing with Python, Bird, Steven